# Automated Library Management System

This notebook provides an interface for managing the library system, including:
- Book Management
- Member Management
- Loan Management
- Data Analysis and Reporting

In [ ]:
# Import required libraries
import sys
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from models import Book, Member, Loan
from database import engine, db_session, init_db

# Initialize database
init_db()

## Book Management Functions

In [ ]:
def add_book(title, author, isbn, publish_date, description, cover_image=None):
    """Add a new book to the library"""
    try:
        if isinstance(publish_date, str):
            publish_date = datetime.strptime(publish_date, '%Y-%m-%d')
            
        book = Book(
            title=title,
            author=author,
            isbn=isbn,
            publish_date=publish_date,
            description=description,
            cover_image=cover_image or 'https://images.pexels.com/photos/1005324/literature-book-open-pages-1005324.jpeg',
            available=True
        )
        db_session.add(book)
        db_session.commit()
        return f"Book '{title}' added successfully!"
    except Exception as e:
        db_session.rollback()
        return f"Error adding book: {str(e)}"

def search_books(query):
    """Search books by title, author, or ISBN"""
    books = Book.query.filter(
        (Book.title.ilike(f'%{query}%')) |
        (Book.author.ilike(f'%{query}%')) |
        (Book.isbn.ilike(f'%{query}%'))
    ).all()
    
    return pd.DataFrame([
        {
            'ID': book.id,
            'Title': book.title,
            'Author': book.author,
            'ISBN': book.isbn,
            'Available': book.available
        } for book in books
    ])

def update_book(book_id, **kwargs):
    """Update book information"""
    try:
        book = Book.query.get(book_id)
        if not book:
            return "Book not found"
            
        for key, value in kwargs.items():
            if hasattr(book, key):
                setattr(book, key, value)
                
        db_session.commit()
        return f"Book '{book.title}' updated successfully!"
    except Exception as e:
        db_session.rollback()
        return f"Error updating book: {str(e)}"

## Member Management Functions

In [ ]:
def add_member(name, email):
    """Add a new member to the library"""
    try:
        member = Member(
            name=name,
            email=email,
            join_date=datetime.utcnow()
        )
        db_session.add(member)
        db_session.commit()
        return f"Member '{name}' added successfully!"
    except Exception as e:
        db_session.rollback()
        return f"Error adding member: {str(e)}"

def search_members(query):
    """Search members by name or email"""
    members = Member.query.filter(
        (Member.name.ilike(f'%{query}%')) |
        (Member.email.ilike(f'%{query}%'))
    ).all()
    
    return pd.DataFrame([
        {
            'ID': member.id,
            'Name': member.name,
            'Email': member.email,
            'Join Date': member.join_date.strftime('%Y-%m-%d')
        } for member in members
    ])

def get_member_loans(member_id):
    """Get all loans for a specific member"""
    member = Member.query.get(member_id)
    if not member:
        return "Member not found"
        
    loans = pd.DataFrame([
        {
            'Book Title': loan.book.title,
            'Loan Date': loan.loan_date.strftime('%Y-%m-%d'),
            'Return Date': loan.return_date.strftime('%Y-%m-%d') if loan.return_date else 'Not returned'
        } for loan in member.loans
    ])
    
    return loans

## Loan Management Functions

In [ ]:
def issue_book(book_id, member_id):
    """Issue a book to a member"""
    try:
        book = Book.query.get(book_id)
        member = Member.query.get(member_id)
        
        if not book or not member:
            return "Book or member not found"
            
        if not book.available:
            return "Book is not available"
            
        loan = Loan(
            book_id=book_id,
            member_id=member_id,
            loan_date=datetime.utcnow()
        )
        
        book.available = False
        db_session.add(loan)
        db_session.commit()
        
        return f"Book '{book.title}' issued to {member.name}"
    except Exception as e:
        db_session.rollback()
        return f"Error issuing book: {str(e)}"

def return_book(book_id):
    """Return a book to the library"""
    try:
        book = Book.query.get(book_id)
        if not book:
            return "Book not found"
            
        loan = Loan.query.filter_by(book_id=book_id, return_date=None).first()
        if not loan:
            return "No active loan found for this book"
            
        loan.return_date = datetime.utcnow()
        book.available = True
        db_session.commit()
        
        return f"Book '{book.title}' returned successfully"
    except Exception as e:
        db_session.rollback()
        return f"Error returning book: {str(e)}"

def get_overdue_loans(days_overdue=14):
    """Get all overdue loans"""
    overdue_date = datetime.utcnow() - timedelta(days=days_overdue)
    overdue_loans = Loan.query.filter(
        Loan.return_date.is_(None),
        Loan.loan_date < overdue_date
    ).all()
    
    return pd.DataFrame([
        {
            'Book Title': loan.book.title,
            'Member Name': loan.member.name,
            'Loan Date': loan.loan_date.strftime('%Y-%m-%d'),
            'Days Overdue': (datetime.utcnow() - loan.loan_date).days
        } for loan in overdue_loans
    ])

## Data Analysis and Reporting Functions

In [ ]:
def get_library_statistics():
    """Get general statistics about the library"""
    total_books = Book.query.count()
    available_books = Book.query.filter_by(available=True).count()
    total_members = Member.query.count()
    active_loans = Loan.query.filter_by(return_date=None).count()
    
    stats = {
        'Total Books': total_books,
        'Available Books': available_books,
        'Total Members': total_members,
        'Active Loans': active_loans
    }
    
    return pd.Series(stats)

def get_popular_books(limit=10):
    """Get most frequently borrowed books"""
    popular_books = db_session.query(
        Book.title,
        Book.author,
        func.count(Loan.id).label('loan_count')
    ).join(Loan).group_by(Book.id).order_by(func.count(Loan.id).desc()).limit(limit).all()
    
    return pd.DataFrame([
        {
            'Title': title,
            'Author': author,
            'Times Borrowed': loan_count
        } for title, author, loan_count in popular_books
    ])

def get_member_activity(limit=10):
    """Get most active members"""
    active_members = db_session.query(
        Member.name,
        Member.email,
        func.count(Loan.id).label('loan_count')
    ).join(Loan).group_by(Member.id).order_by(func.count(Loan.id).desc()).limit(limit).all()
    
    return pd.DataFrame([
        {
            'Name': name,
            'Email': email,
            'Books Borrowed': loan_count
        } for name, email, loan_count in active_members
    ])

## Example Usage

In [ ]:
# Add a new book
print(add_book(
    title="The Great Gatsby",
    author="F. Scott Fitzgerald",
    isbn="9780743273565",
    publish_date="1925-04-10",
    description="The story of the mysteriously wealthy Jay Gatsby and his love for the beautiful Daisy Buchanan."
))

# Add a new member
print(add_member(
    name="John Doe",
    email="john.doe@example.com"
))

# Search for books
print("\nSearching for 'Gatsby':")
print(search_books('Gatsby'))

# Get library statistics
print("\nLibrary Statistics:")
print(get_library_statistics())